In [ ]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np
import wide_residual_network as wrn
import keras.callbacks as callbacks

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [ ]:
# Configure logs
import logging

# Record settings
LOG_FORMAT = "%(levelname)s %(asctime)s - %(message)s"
logging.basicConfig(filename="wrn_10_2_systematically.txt",format = LOG_FORMAT, level = logging.DEBUG, filemode = "a")
logs = logging.getLogger()

In [ ]:
def swish(x):
    return x*K.sigmoid(x)

def swish_125(x):
    return 1.125*x*K.sigmoid(x)

def swish_250(x):
    return 1.25*x*K.sigmoid(x)

def swish_375(x):
    return 1.375*x*K.sigmoid(x)

def swish_500(x):
    return 1.5*x*K.sigmoid(x)

def swish_625(x):
    return 1.625*x*K.sigmoid(x)

def swish_750(x):
    return 1.75*x*K.sigmoid(x)

def swish_875(x):
    return 1.875*x*K.sigmoid(x)

def swish_200(x):
    return 2*x*K.sigmoid(x)

In [ ]:
activations = ["relu", swish, swish_125, swish_250, swish_375, swish_500, swish_625, swish_750, swish_875, swish_200]
names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

In [ ]:
#data augmentation
datagen = ImageDataGenerator(width_shift_range=0.125,
                             height_shift_range=0.125,
                             horizontal_flip=True,
                             fill_mode="reflect")
datagen.fit(x_train)

In [ ]:
def schedule(x):
    if x < 45:
        return 0.125
    elif x < 85:
        return 0.125*0.2
    elif x < 105:
        return 0.125*0.2*0.2
    elif x < 125:
        return 0.125*0.2*0.2*0.2
    else:
        return 0.125*0.2*0.2*0.2*0.5

In [ ]:
def run(act):# SWISH - 125
    init_shape = (3, 32, 32) if K.image_dim_ordering() == 'th' else (32, 32, 3)
    # For WRN-16-8 put N = 2, k = 8
    # For WRN-28-10 put N = 4, k = 10
    # For WRN-40-4 put N = 6, k = 4
    model = wrn.create_wide_residual_network(act, init_shape, nb_classes=num_classes, N=1, k=2)

    print("Model Created")
    batch_size  = 128
    epochs = 125

    opt = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.0, nesterov=False)
    lr_1 = keras.callbacks.LearningRateScheduler(schedule)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    print("Finished compiling")

    ####################
    # Network training #
    ####################

    print("Gonna fit the model")
    his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
#     print(his.history)
    return his.history

In [ ]:
for i, act in enumerate(activations):
    logs.info("\n \n Starting training for "+names[i])
    first = run(act)
    logs.info("First: "+str(first["val_acc"][-1]))
    second = run(act)
    logs.info("Second: "+str(second["val_acc"][-1]))
    third = run(act)
    logs.info("Third: "+str(third["val_acc"][-1]))
    print("\n \n \n \n")

In [ ]:
new_activations = activations[-2:]
new_names = names[-2:]
for i, act in enumerate(new_activations):
    logs.info("\n \n Starting training for "+new_names[i])
    first = run(act)
    logs.info("First: "+str(first["val_acc"][-1]))
    second = run(act)
    logs.info("Second: "+str(second["val_acc"][-1]))
    third = run(act)
    logs.info("Third: "+str(third["val_acc"][-1]))
    print("\n \n \n \n")